In [ ]:
# Extract all features from brain MRIs

import cv2
import os
import sys
import glob
import csv
import imageio
from skimage import color
from skimage.transform import resize
from scipy import ndimage
from scipy.stats import kurtosis, skew
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.color import label2rgb
from skimage.filters.rank import entropy
from skimage.morphology import disk
import mahotas as mt
import pywt
import pywt.data
from numpy.testing import assert_array_almost_equal

np.set_printoptions(threshold=sys.maxsize)

q = "'"

# Image folder path
path = '...'
path_out = '...'
list_img = os.listdir(path) 
# number_files = number of files in the input folder
number_files = len(list_img)
print(number_files)

i = 0
j = 1
# Input image folder is 'yes'
features = [[0 for x in range(21)] for y in range(number_files+1)]
features[0][0] = 'Image'
features[0][1] = 'Mean'
features[0][2] = 'Variance'
features[0][3] = 'Skewness'
features[0][4] = 'Kurtosis'
features[0][5] = 'LL-DWT'
features[0][6] = 'LH-DWT'
features[0][7] = 'HL-DWT'
features[0][8] = 'HH-DWT'
features[0][9] = 'DWT-Coefficient'
features[0][10] = 'Cooccurance Matrix'
features[0][11] = 'Contrast' 
features[0][12] = 'Dissimilarity'
features[0][13] = 'Homogeneity'
features[0][14] = 'ASM'
features[0][15] = 'Energy'
features[0][16] = 'Correlation'
features[0][17] = 'Entropy'
features[0][18] = 'LBP'
features[0][19] = 'Haralick'
features[0][20] = 'Label'


for file in glob.glob(path + '*.jpg'): 
    i = i+1
    print("File - ", i)
    head, tail = os.path.split(file)
    #print(head)
    #print(tail)
    # f_name has the file name with extension i.e. Y1.jpg, Y2.jpg
    f_name = tail
    features[j][0] = tail
    head2, tail2 = f_name.split(".")
    #print(head2)
    # filename has only the name i.e. Y1, Y2
    filename = head2
    #print(filename)
    
    
    # Read each image in im
    im = imageio.imread(file) 
    
    # Convert im from 3D to 2D
    image = color.rgb2gray(im)

    # Resize each image into height = 300 and width = 250
    img_resized = resize(image, (256, 256))
    
    #------- Intensity Features --------------------------------
    mean = ndimage.mean(img_resized)
    variance = ndimage.variance(img_resized)
    skewness = skew(img_resized, axis=None)
    kurtos = kurtosis(img_resized, axis=None)
    
    #------- 2D DWT Features --------------------------------
    coeffs2 = pywt.dwt2(img_resized, 'bior1.3')
    LL, (LH, HL, HH) = coeffs2
    coeffs = pywt.wavedecn(img_resized, wavelet='db2', level=3)
    arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    
    #------- Texture Features --------------------------------
    img = np.array(img_resized, dtype=np.uint8)

    # -------------- Cooccurance Matrix --------------------------
    g = greycomatrix(img, [1, 2, 3], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, normed=True, symmetric=True)
    
    contrast = greycoprops(g, 'contrast')
    dis = greycoprops(g, 'dissimilarity')
    hom = greycoprops(g, 'homogeneity')
    asm = greycoprops(g, 'ASM')
    ene = greycoprops(g, 'energy')
    cor = greycoprops(g, 'correlation')
    
    # --------------- Entropy ---------------------
    entr = entropy(img_resized, disk(10))
    
    
    # -------------- Local Binary Pattern--------------------------
    # settings for LBP
    radius = 3
    n_points = 8 * radius
    METHOD = 'uniform'
    lbp = local_binary_pattern(img_resized, n_points, radius, METHOD)
    
    
    # ------------------- Haralick Texture Feature Vector ---------------
    img = mt.imread(file) 
     
    # filtering the imagwe 
    img = img[:, :] 
      
    # setting gaussian filter 
    gaussian = mt.gaussian_filter(img, 15) 
   
    # setting threshold value 
    gaussian = (gaussian > gaussian.mean()) 
   
    # making is labeled image 
    labeled, n = mt.label(gaussian) 
    
    h_feature = mt.features.haralick(labeled) 
     
       
    features[j][1] = mean
    features[j][2] = variance
    features[j][3] = skewness
    features[j][4] = kurtos
    
    features[j][5] = np.mean(LL)
    features[j][6] = np.mean(LH)
    features[j][7] = np.mean(HL)
    features[j][8] = np.mean(HH)
    features[j][9] = np.mean(arr)
    
    features[j][10] = np.mean(g)
    features[j][11] = np.mean(contrast)
    features[j][12] = np.mean(dis)
    features[j][13] = np.mean(hom)
    features[j][14] = np.mean(asm)
    features[j][15] = np.mean(ene)
    features[j][16] = np.mean(cor)  
    
    features[j][17] = np.mean(entr)
    
    features[j][18] = np.mean(lbp)
    
    features[j][19] = np.mean(h_feature)
    
    if (filename[0] == 'Y'):
        features[j][20] = 1   
    elif (filename[0] == 'N'):
        features[j][20] = 0
    
    
    j = j + 1


with open(path_out + 'All_Features.csv', 'wt', newline ='') as file:
    writer1 = csv.writer(file, delimiter=',')
    for x in features:
        writer1.writerow(x)
file.close()